In [ ]:
import math
import scipy.io
from scipy.sparse import coo_matrix
import pandas as pd
import matplotlib.pyplot as plt
import time
import cv2
import numpy as np

from sys import getsizeof

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler

from MyDataset import MyDataset
import random

from statsmodels.stats.proportion import proportion_confint

import optuna
from optuna.trial import TrialState

print(torch.__version__)

In [ ]:
torch.set_default_dtype(torch.float64)
torch.manual_seed(10) # set seed before creating model

images = torch.load('256x256_images_50_percent.pt')
labels = torch.load('256x256_labels_50_percent.pt')

In [ ]:
# number of epochs to train the model
##n_epochs = 30
# Learning rate of optimizer
##learning_rate = 0.0001
# Batch size of data loaders and batch size used when training model
batch_size = 64
#dropout rate
##dropout = 0.30

In [ ]:
print(len(images))
print(len(labels))

random.Random(10).shuffle(images) # shuffling with seed
random.Random(10).shuffle(labels) 

size = len(images)

dataset = MyDataset(images,labels)

split_indices = list(range(0,size))

train_idx=split_indices[0:round(0.70*size)]
val_idx=split_indices[round(0.70*size):round(0.85*size)]
test_idx=split_indices[round(0.85*size):]
print(train_idx)
print(val_idx)
print(test_idx)

train_dataset=MyDataset([images[i] for i in train_idx],[labels[i] for i in train_idx])
val_dataset=MyDataset([images[i] for i in val_idx],[labels[i] for i in val_idx])
test_dataset=MyDataset([images[i] for i in test_idx],[labels[i] for i in test_idx])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

for data, target in train_loader:
    print(data.shape)
    print(target.shape)
    

In [ ]:
# define the CNN architecture
class Net(nn.Module):
    def __init__(self, layerAmount, pooling_stride, dropout, startFilter):
        super(Net, self).__init__()

        self.conv=nn.ModuleList()
        self.conv_bn=nn.ModuleList()
        
        self.layerAmount=layerAmount
        
        self.kernel_size = 3
        self.padding = 1
        self.stride = 1


        self.conv.append(nn.Conv2d(1, startFilter, self.kernel_size, 1, self.padding))
        self.conv_bn.append(nn.BatchNorm2d(startFilter))
        
        #The layer computation here might not be 100% correct but it atleast calculates correctly for 3-6 layers
        for i in range(1, self.layerAmount):
            self.conv.append(nn.Conv2d(startFilter*(2**(i-1)), startFilter*(2**i), self.kernel_size, 1, self.padding))
            self.conv_bn.append(nn.BatchNorm2d(startFilter*(2**i)))
            
        x=256
        y=256
        d=startFilter*(2**(self.layerAmount-1))
        
        for i in range(0, self.layerAmount):
            x=(x-self.kernel_size+(2*self.padding))/pooling_stride[0]
            y=(y-self.kernel_size+(2*self.padding))/pooling_stride[1]
            #print(math.ceil(x))
            #print(math.ceil(y))
            #print('')
            
        ##
        
            
        self.features = math.ceil(x) * math.ceil(y) * math.ceil(d)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=pooling_stride)
        
        # linear layer (X -> 10)
        self.fc1 = nn.Linear(self.features, 10)
        # linear layer (10 -> 1)
        self.fc2 = nn.Linear(10, 1)
        # dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        
        for i in range(0, self.layerAmount):
            x = self.conv[i](x) #convolution
            x = self.conv_bn[i](x) # batch normalization
            x = F.relu(x) # reLU
            x = self.pool(x) # pooling
            #print(x.shape)
        
        x = x.view(-1, self.features)
        #print(x.shape)
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        #print(x.shape)
        # add dropout layer
        x = self.dropout(x)
        # output
        x = torch.sigmoid(self.fc2(x))
        x = x.view(x.size(0))
        #print(x.shape)    
        return x  

In [ ]:
train_on_gpu = torch.cuda.is_available()

def objective(trial):
    n_epochs = trial.suggest_int("n_epochs", 30, 70)
    dropout_p = trial.suggest_float("dropout_p", 0.0, 0.7)
    n_layers = trial.suggest_int("n_layers", 3, 6)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True)
    #x_stride = trial.suggest_int("x_stride", 2, 3)
    #y_stride = trial.suggest_int("y_stride", 2, 3)
    
    model = Net(layerAmount=n_layers, pooling_stride=[2, 2], dropout=dropout_p, startFilter=8)
    
    # specify loss function (Binary cross entropy)
    criterion = nn.BCELoss()

    # specify optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    if not train_on_gpu:
        print('CUDA is not available.  Training on CPU ...')
    else:
        model.cuda()
        criterion.cuda()
        print('CUDA is available!  Training on GPU ...')
        
    valid_loss_min = np.Inf # track change in validation loss

    train_loss= [0.0] * n_epochs
    valid_loss= [0.0] * n_epochs

    max_accuracy = 0
    
    for epoch in range(0, n_epochs):

        # keep track of training and validation loss
        #train_loss[epoch] = 0.0
        #valid_loss[epoch] = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        for data, target in train_loader:
            data=data.to_dense() # model needs dense matrices as input
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            output = output.to(torch.float64) #
            target = target.to(torch.float64) #
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss[epoch] += loss.item()*data.size(0)

        ######################    
        # validate the model #
        ######################     
        total = 0
        true_positive = 0
        true_negative = 0
        
        model.eval()
        for data, target in valid_loader:
            data=data.to_dense() # model needs dense matrices as input
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            output = output.to(torch.float64) #
            target = target.to(torch.float64) #
            #print(output)
            #print(target)
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss[epoch] += loss.item()*data.size(0)
            
            for i in range(len(output)):
                if (target[i] == 1) and (output[i] >= 0.5):
                    true_positive += 1
                elif (target[i] == 0) and (output[i] < 0.5):
                    true_negative += 1
                total +=1

        # calculate average losses
        train_loss[epoch] = train_loss[epoch]/len(train_loader.sampler)
        valid_loss[epoch] = valid_loss[epoch]/len(valid_loader.sampler)

        accuracy = (true_positive + true_negative)/total
        
        if (accuracy > max_accuracy):
            max_accuracy = accuracy
            
        print('accuracy:' + str(accuracy) + ', best epoch:' + str(max_accuracy))
        
        trial.report(max_accuracy, epoch+1)
        
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
            
    return max_accuracy
    

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, timeout=None, show_progress_bar=True)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
print('hello')